In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/semeval-2014-task-4-aspectbasedsentimentanalysis/Laptop_Train_v2.xml
/kaggle/input/semeval-2014-task-4-aspectbasedsentimentanalysis/Restaurants_Test_Data_PhaseB.csv
/kaggle/input/semeval-2014-task-4-aspectbasedsentimentanalysis/laptops-trial.csv
/kaggle/input/semeval-2014-task-4-aspectbasedsentimentanalysis/Laptops_Test_Data_PhaseB.csv
/kaggle/input/semeval-2014-task-4-aspectbasedsentimentanalysis/restaurants-trial.xml
/kaggle/input/semeval-2014-task-4-aspectbasedsentimentanalysis/SemEvalSchema.xsd
/kaggle/input/semeval-2014-task-4-aspectbasedsentimentanalysis/Laptops_Test_Data_phaseB.xml
/kaggle/input/semeval-2014-task-4-aspectbasedsentimentanalysis/Laptops_Test_Data_PhaseA.xml
/kaggle/input/semeval-2014-task-4-aspectbasedsentimentanalysis/Laptops_Test_Data_PhaseA.csv
/kaggle/input/semeval-2014-task-4-aspectbasedsentimentanalysis/Restaurants_Train_v2.csv
/kaggle/input/semeval-2014-task-4-aspectbasedsentimentanalysis/laptops-trial.xml
/kaggle/input/semeval-2014-task-4-asp

In [2]:
import os

# Duyệt qua tất cả các file trong thư mục /kaggle/input
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if 'laptop' in filename.lower():  
            print(os.path.join(dirname, filename))

/kaggle/input/semeval-2014-task-4-aspectbasedsentimentanalysis/Laptop_Train_v2.xml
/kaggle/input/semeval-2014-task-4-aspectbasedsentimentanalysis/laptops-trial.csv
/kaggle/input/semeval-2014-task-4-aspectbasedsentimentanalysis/Laptops_Test_Data_PhaseB.csv
/kaggle/input/semeval-2014-task-4-aspectbasedsentimentanalysis/Laptops_Test_Data_phaseB.xml
/kaggle/input/semeval-2014-task-4-aspectbasedsentimentanalysis/Laptops_Test_Data_PhaseA.xml
/kaggle/input/semeval-2014-task-4-aspectbasedsentimentanalysis/Laptops_Test_Data_PhaseA.csv
/kaggle/input/semeval-2014-task-4-aspectbasedsentimentanalysis/laptops-trial.xml
/kaggle/input/semeval-2014-task-4-aspectbasedsentimentanalysis/Laptop_Train_v2.csv


In [3]:
import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if 'laptop' in filename.lower():  
            if 'csv' in filename:
                print(os.path.join(dirname, filename))

/kaggle/input/semeval-2014-task-4-aspectbasedsentimentanalysis/laptops-trial.csv
/kaggle/input/semeval-2014-task-4-aspectbasedsentimentanalysis/Laptops_Test_Data_PhaseB.csv
/kaggle/input/semeval-2014-task-4-aspectbasedsentimentanalysis/Laptops_Test_Data_PhaseA.csv
/kaggle/input/semeval-2014-task-4-aspectbasedsentimentanalysis/Laptop_Train_v2.csv


In [4]:
import pandas as pd
train_df = pd.read_csv('/kaggle/input/semeval-2014-task-4-aspectbasedsentimentanalysis/Laptop_Train_v2.csv')
train_df

,id,Sentence,Aspect Term,polarity,from,to
0,2339,I charge it at night and skip taking the cord ...,cord,neutral,41,45
1,2339,I charge it at night and skip taking the cord ...,battery life,positive,74,86
2,1316,The tech guy then said the service center does...,service center,negative,27,41
3,1316,The tech guy then said the service center does...,"""sales"" team",negative,109,121
4,1316,The tech guy then said the service center does...,tech guy,neutral,4,12
...,...,...,...,...,...,...
2353,2272,We also use Paralles so we can run virtual mac...,Windows Server Enterprise 2003,neutral,104,134
2354,2272,We also use Paralles so we can run virtual mac...,Windows Server 2008 Enterprise,neutral,140,170
2355,848,"How Toshiba handles the repair seems to vary, ...",repair,conflict,24,30
2356,848,"How Toshiba handles the repair seems to vary, ...",repair,positive,130,136


# Model phat hien

## Tiền Xử lí 

In [5]:

from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer

2025-09-18 12:30:41.455649: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758198641.624701      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758198641.679535      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
from datasets import Dataset

In [7]:
from transformers import AutoTokenizer
from datasets import Dataset
tokenizer_name="bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
def df_to_hf_dataset(df, tokenizer):
    
    bio_data = []
    grouped = df.groupby('id', sort=False)

    for _, group in grouped:
        sentence = group['Sentence'].iloc[0]
        
        aspects = [(int(row['from']), int(row['to'])) for _, row in group.iterrows()]
        encoding = tokenizer(sentence, return_offsets_mapping=True, truncation=True)
        tokens = tokenizer.convert_ids_to_tokens(encoding['input_ids'])
        offsets = encoding['offset_mapping']
        labels = [0] * len(tokens)
        for start, end in aspects:
            for i, (s, e) in enumerate(offsets):
                if e > start and s < end:  
                    if s <= start:
                        labels[i] = 1
                    else:
                        labels[i] = 2

        bio_data.append({'tokens': tokens, 'labels': labels})
    dataset = Dataset.from_list(bio_data)
    return dataset


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [8]:
dataset = df_to_hf_dataset(train_df,tokenizer)

In [9]:
dataset

Dataset({
    features: ['tokens', 'labels'],
    num_rows: 1488
})

In [10]:
train_df

,id,Sentence,Aspect Term,polarity,from,to
0,2339,I charge it at night and skip taking the cord ...,cord,neutral,41,45
1,2339,I charge it at night and skip taking the cord ...,battery life,positive,74,86
2,1316,The tech guy then said the service center does...,service center,negative,27,41
3,1316,The tech guy then said the service center does...,"""sales"" team",negative,109,121
4,1316,The tech guy then said the service center does...,tech guy,neutral,4,12
...,...,...,...,...,...,...
2353,2272,We also use Paralles so we can run virtual mac...,Windows Server Enterprise 2003,neutral,104,134
2354,2272,We also use Paralles so we can run virtual mac...,Windows Server 2008 Enterprise,neutral,140,170
2355,848,"How Toshiba handles the repair seems to vary, ...",repair,conflict,24,30
2356,848,"How Toshiba handles the repair seems to vary, ...",repair,positive,130,136


In [11]:
train_df['Sentence'][0]

'I charge it at night and skip taking the cord with me because of the good battery life.'

In [12]:
print(dataset['tokens'][0])

['[CLS]', 'i', 'charge', 'it', 'at', 'night', 'and', 'skip', 'taking', 'the', 'cord', 'with', 'me', 'because', 'of', 'the', 'good', 'battery', 'life', '.', '[SEP]']


In [13]:
print(dataset['labels'][0])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0]


In [14]:
import torch

In [15]:
import torch

def prepare_dataset_for_bert(dataset, tokenizer):
    input_ids_list = []
    attention_mask_list = []
    labels_list = []

    for tokens, labels in zip(dataset['tokens'], dataset['labels']):
        input_ids = tokenizer.convert_tokens_to_ids(tokens)
        attention_mask = [1] * len(input_ids)

        label_ids = labels.copy()
        # gán -100 cho [CLS] và [SEP]
        label_ids[0] = -100
        label_ids[-1] = -100

       
        input_ids_list.append(torch.tensor(input_ids))
        attention_mask_list.append(torch.tensor(attention_mask))
        labels_list.append(torch.tensor(label_ids))

    return input_ids_list, attention_mask_list, labels_list


In [16]:
from transformers import AutoTokenizer
input_ids_list, attention_mask_list, labels_list = prepare_dataset_for_bert(dataset, tokenizer)

In [17]:
print(dataset['tokens'][0])
print(len(dataset['tokens'][0]))

['[CLS]', 'i', 'charge', 'it', 'at', 'night', 'and', 'skip', 'taking', 'the', 'cord', 'with', 'me', 'because', 'of', 'the', 'good', 'battery', 'life', '.', '[SEP]']
21


In [18]:
# Kiểm tra
print(input_ids_list[0])
print(attention_mask_list[0])
print(labels_list[0])

print(len(input_ids_list[0]))
print(len(attention_mask_list[0]))
print(len(labels_list[0]))


tensor([  101,  1045,  3715,  2009,  2012,  2305,  1998, 13558,  2635,  1996,
        11601,  2007,  2033,  2138,  1997,  1996,  2204,  6046,  2166,  1012,
          102])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
tensor([-100,    0,    0,    0,    0,    0,    0,    0,    0,    0,    1,    0,
           0,    0,    0,    0,    0,    1,    2,    0, -100])
21
21
21


In [19]:
hf_dataset = Dataset.from_dict({
    'input_ids': input_ids_list,
    'attention_mask': attention_mask_list,
    'labels': labels_list
})

## Tạo các tập

In [20]:
from transformers import AutoTokenizer
from datasets import Dataset
tokenizer_name="bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

In [21]:
path_train = "/kaggle/input/semeval-2014-task-4-aspectbasedsentimentanalysis/Laptop_Train_v2.csv"
path_test = "/kaggle/input/semeval-2014-task-4-aspectbasedsentimentanalysis/laptops-trial.csv"

In [22]:
df_train = pd.read_csv(path_train)
df_test = pd.read_csv(path_test)

In [23]:
df_train.head()

,id,Sentence,Aspect Term,polarity,from,to
0,2339,I charge it at night and skip taking the cord ...,cord,neutral,41,45
1,2339,I charge it at night and skip taking the cord ...,battery life,positive,74,86
2,1316,The tech guy then said the service center does...,service center,negative,27,41
3,1316,The tech guy then said the service center does...,"""sales"" team",negative,109,121
4,1316,The tech guy then said the service center does...,tech guy,neutral,4,12


In [24]:
df_test.head()

,id,Sentence,Aspect Term,polarity,from,to
0,2128,I liked the aluminum body.,aluminum body,positive,12,25
1,81,Lightweight and the screen is beautiful!,screen,positive,20,26
2,353,"From the build quality to the performance, eve...",build quality,negative,9,22
3,353,"From the build quality to the performance, eve...",performance,negative,30,41
4,655,It was truly a great computer costing less tha...,costing,positive,30,37


In [25]:
dataset_train = df_to_hf_dataset(df_train,tokenizer)
input_ids_list_train, attention_mask_list_train, labels_list_train = prepare_dataset_for_bert(dataset_train, tokenizer)
train_datset = Dataset.from_dict({
    'input_ids': input_ids_list_train,
    'attention_mask': attention_mask_list_train,
    'labels': labels_list_train
})

In [26]:
dataset_test = df_to_hf_dataset(df_test,tokenizer)
input_ids_list_test, attention_mask_list_test, labels_list_test = prepare_dataset_for_bert(dataset_test, tokenizer)
test_datset = Dataset.from_dict({
    'input_ids': input_ids_list_test,
    'attention_mask': attention_mask_list_test,
    'labels': labels_list_test
})

## lấy model 

In [27]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification

In [28]:
model = AutoModelForTokenClassification.from_pretrained("bert-base-uncased", num_labels=3)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
model

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [30]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer, padding=True)

In [31]:
! pip install seqeval

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=90e1c151fd5699e4977f93b896cf0762f6106aaedd02afa5c4ee4c9edf96726e
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval


In [32]:
from seqeval.metrics import precision_score, recall_score, f1_score
import numpy as np

id2label = {0:'O', 1:'B-ASP', 2:'I-ASP'}
def compute_metrics(p):
    # p.predictions: (batch_size, seq_len, num_labels)
    # p.label_ids: (batch_size, seq_len)
    preds = np.argmax(p.predictions, axis=-1) # batch_size, seq_len
    labels = p.label_ids

    true_labels = []
    true_preds = []

    for pred_seq, label_seq in zip(preds, labels):
        pred_labels = []
        gold_labels = []
        for p_lab, l_lab in zip(pred_seq, label_seq):
            if l_lab == -100:
                continue 
            pred_labels.append(id2label[p_lab])
            gold_labels.append(id2label[l_lab])
        true_preds.append(pred_labels)
        true_labels.append(gold_labels)

    precision = precision_score(true_labels, true_preds)
    recall = recall_score(true_labels, true_preds)
    f1 = f1_score(true_labels, true_preds)

    return {
        "precision": precision,
        "recall": recall,
        "f1": f1
    }


In [33]:
training_args = TrainingArguments(
    output_dir="./bert_aspect_model",
    num_train_epochs=10,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100, 
    eval_steps=100,              
    save_total_limit=2,
    save_steps=100,
    report_to="none",
    disable_tqdm=False
)


In [34]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_datset,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)


In [35]:
trainer.train()

Step,Training Loss
100,0.147000
200,0.019100
300,0.006200
400,0.002500


TrainOutput(global_step=470, training_loss=0.037465677711557836, metrics={'train_runtime': 100.1352, 'train_samples_per_second': 148.599, 'train_steps_per_second': 4.694, 'total_flos': 433846359284832.0, 'train_loss': 0.037465677711557836, 'epoch': 10.0})

In [36]:
trainer.evaluate(eval_dataset=train_datset)

{'eval_loss': 0.0004915378522127867,
 'eval_precision': 0.9995759117896522,
 'eval_recall': 0.9995759117896522,
 'eval_f1': 0.9995759117896522,
 'eval_runtime': 2.7344,
 'eval_samples_per_second': 544.188,
 'eval_steps_per_second': 17.189,
 'epoch': 10.0}

In [37]:
trainer.evaluate(eval_dataset=test_datset)

{'eval_loss': 0.07412030547857285,
 'eval_precision': 0.9215686274509803,
 'eval_recall': 0.9591836734693877,
 'eval_f1': 0.9400000000000001,
 'eval_runtime': 0.0669,
 'eval_samples_per_second': 597.745,
 'eval_steps_per_second': 29.887,
 'epoch': 10.0}

## Test

In [38]:
import torch

def predict_aspect(sentence, model, tokenizer, id2label):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    
    # Tokenize và chuyển sang device
    encoding = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1).squeeze().tolist()
    
    tokens = tokenizer.convert_ids_to_tokens(input_ids.squeeze().cpu())
    labels = [id2label[pred] for pred in predictions]
    
    return list(zip(tokens, labels))

In [39]:
sentence = "The battery life of this laptop is amazing."
result = predict_aspect(sentence, model, tokenizer, id2label)

for token, label in result:
    print(f"{token}\t{label}")

[CLS]	O
the	O
battery	B-ASP
life	I-ASP
of	O
this	O
laptop	O
is	O
amazing	O
.	O
[SEP]	O


In [40]:
sentence = "The battery life is great but the screen is dull and the keyboard feels cheap."
result = predict_aspect(sentence, model, tokenizer, id2label)

for token, label in result:
    print(f"{token}\t{label}")

[CLS]	O
the	O
battery	B-ASP
life	I-ASP
is	O
great	O
but	O
the	O
screen	B-ASP
is	O
dull	O
and	O
the	O
keyboard	B-ASP
feels	O
cheap	O
.	O
[SEP]	O


## Lưu 

In [41]:
save_path = "./bert_aspect_model_saved"

model.save_pretrained(save_path)

tokenizer.save_pretrained(save_path)

('./bert_aspect_model_saved/tokenizer_config.json',
 './bert_aspect_model_saved/special_tokens_map.json',
 './bert_aspect_model_saved/vocab.txt',
 './bert_aspect_model_saved/added_tokens.json',
 './bert_aspect_model_saved/tokenizer.json')

In [42]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(save_path)
tokenizer = AutoTokenizer.from_pretrained(save_path)


# Đánh giá khía cạnh 

## Tiền Xữ lí 

In [43]:
df_train = pd.read_csv(path_train)
df_test = pd.read_csv(path_test)

In [44]:
df_train.head()

,id,Sentence,Aspect Term,polarity,from,to
0,2339,I charge it at night and skip taking the cord ...,cord,neutral,41,45
1,2339,I charge it at night and skip taking the cord ...,battery life,positive,74,86
2,1316,The tech guy then said the service center does...,service center,negative,27,41
3,1316,The tech guy then said the service center does...,"""sales"" team",negative,109,121
4,1316,The tech guy then said the service center does...,tech guy,neutral,4,12


In [45]:
df_train_sent = df_train[['Sentence', 'Aspect Term', 'polarity']].copy()
df_test_sent = df_test[['Sentence', 'Aspect Term', 'polarity']].copy()

In [46]:
label2id = {"negative":0, "neutral":1, "positive":2}

In [47]:
id2label = {v:k for k,v in label2id.items()}

In [48]:
df_train_sent['label'] = df_train_sent['polarity'].map(label2id)
df_test_sent['label'] = df_test_sent['polarity'].map(label2id)

In [49]:
df_train_sent.head()

,Sentence,Aspect Term,polarity,label
0,I charge it at night and skip taking the cord ...,cord,neutral,1.0
1,I charge it at night and skip taking the cord ...,battery life,positive,2.0
2,The tech guy then said the service center does...,service center,negative,0.0
3,The tech guy then said the service center does...,"""sales"" team",negative,0.0
4,The tech guy then said the service center does...,tech guy,neutral,1.0


In [50]:
print(df_train_sent['label'].isnull().sum())
print(df_test_sent['label'].isnull().sum())


45
0


In [51]:
df_train_sent = df_train_sent.dropna(subset=['label'])
df_test_sent = df_test_sent.dropna(subset=['label'])

In [52]:
df_train_sent.head()

,Sentence,Aspect Term,polarity,label
0,I charge it at night and skip taking the cord ...,cord,neutral,1.0
1,I charge it at night and skip taking the cord ...,battery life,positive,2.0
2,The tech guy then said the service center does...,service center,negative,0.0
3,The tech guy then said the service center does...,"""sales"" team",negative,0.0
4,The tech guy then said the service center does...,tech guy,neutral,1.0


In [53]:
df_train_sent['label'] = df_train_sent['label'].astype(int)
df_test_sent['label'] = df_test_sent['label'].astype(int)

In [54]:
label_counts = df_train_sent['label'].value_counts()
print(label_counts)

label
2    987
0    866
1    460
Name: count, dtype: int64


In [55]:
dataset_train = Dataset.from_pandas(df_train_sent[['Sentence','Aspect Term','label']])
dataset_test = Dataset.from_pandas(df_test_sent[['Sentence','Aspect Term','label']])

In [56]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [57]:
def tokenize_fn(batch):
    return tokenizer(batch['Aspect Term'], batch['Sentence'], 
                     truncation=True, padding='max_length', max_length=128)

In [58]:
train_dataset = dataset_train.map(tokenize_fn, batched=True)
test_dataset = dataset_test.map(tokenize_fn, batched=True)

Map:   0%|          | 0/2313 [00:00<?, ? examples/s]

Map:   0%|          | 0/49 [00:00<?, ? examples/s]

In [59]:
print(train_dataset['label'].count(None))
print(test_dataset['label'].count(None))

0
0


In [60]:
train_dataset

Dataset({
    features: ['Sentence', 'Aspect Term', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 2313
})

In [61]:
def convert_label_to_int(batch):
    batch['label'] = int(batch['label']) 
    return batch

train_dataset = train_dataset.map(convert_label_to_int)
test_dataset = test_dataset.map(convert_label_to_int)

Map:   0%|          | 0/2313 [00:00<?, ? examples/s]

Map:   0%|          | 0/49 [00:00<?, ? examples/s]

In [62]:
print(train_dataset['Sentence'][0])
print(train_dataset['Aspect Term'][0])
print(train_dataset['label'][0])
print(id2label[train_dataset['label'][0]])
tokens = tokenizer.convert_ids_to_tokens(train_dataset['input_ids'][0])

tokens_no_pad = [t for t, id_ in zip(tokens, train_dataset['input_ids'][0]) if id_ != tokenizer.pad_token_id]

print(tokens_no_pad)

I charge it at night and skip taking the cord with me because of the good battery life.
cord
1
neutral
['[CLS]', 'cord', '[SEP]', 'i', 'charge', 'it', 'at', 'night', 'and', 'skip', 'taking', 'the', 'cord', 'with', 'me', 'because', 'of', 'the', 'good', 'battery', 'life', '.', '[SEP]']


## Model

In [63]:
from transformers import AutoModelForSequenceClassification

num_labels = 3
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [64]:
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer)

In [65]:
training_args = TrainingArguments(
    output_dir="./bert_sentiment_model",
    num_train_epochs=10,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100, 
    eval_steps=100,              
    save_total_limit=2,
    save_steps=100,
    report_to="none",
    disable_tqdm=False
)


In [66]:
! pip install evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 10.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.5.1
    Uninstalling fsspec-2025.5.1:
      Successfully uninstalled fsspec-2025.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvid

In [67]:
import numpy as np
import evaluate  # mới

metric = evaluate.load("accuracy")

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return metric.compute(predictions=preds, references=p.label_ids)


In [68]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)


/tmp/ipykernel_36/2862702860.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [69]:
trainer.train()

Step,Training Loss
100,0.671600
200,0.374500
300,0.226900
400,0.104900
500,0.088800
600,0.042500
700,0.029300


TrainOutput(global_step=730, training_loss=0.2120430154343174, metrics={'train_runtime': 308.8223, 'train_samples_per_second': 74.897, 'train_steps_per_second': 2.364, 'total_flos': 1521453338012160.0, 'train_loss': 0.2120430154343174, 'epoch': 10.0})

In [70]:
trainer.evaluate(eval_dataset=train_dataset)

{'eval_loss': 0.018919624388217926,
 'eval_accuracy': 0.9922178988326849,
 'eval_runtime': 8.2899,
 'eval_samples_per_second': 279.014,
 'eval_steps_per_second': 8.806,
 'epoch': 10.0}

In [71]:
trainer.evaluate(eval_dataset=test_dataset)

{'eval_loss': 0.10492423176765442,
 'eval_accuracy': 0.9795918367346939,
 'eval_runtime': 0.1887,
 'eval_samples_per_second': 259.704,
 'eval_steps_per_second': 10.6,
 'epoch': 10.0}

In [72]:
import torch
from sklearn.metrics import classification_report

def evaluate_model(trainer, test_dataset, id2label):
    """
    trainer: HuggingFace Trainer đã train xong
    test_dataset: HF Dataset dạng torch
    id2label: dict mapping {0:'negative',1:'neutral',2:'positive'}
    """
    model = trainer.model
    model.eval()
    
    input_ids = torch.tensor(test_dataset['input_ids'])
    attention_mask = torch.tensor(test_dataset['attention_mask'])
    labels = torch.tensor(test_dataset['label'])
    
    device = next(model.parameters()).device
    
    preds = []
    true_labels = []
    
    with torch.no_grad():
        for i in range(len(input_ids)):
            inputs = {
                'input_ids': input_ids[i].unsqueeze(0).to(device),
                'attention_mask': attention_mask[i].unsqueeze(0).to(device)
            }
            outputs = model(**inputs)
            logits = outputs.logits
            pred_id = torch.argmax(logits, dim=-1).item()
            
            preds.append(pred_id)
            true_labels.append(labels[i].item() if torch.is_tensor(labels[i]) else labels[i])
    
    true_labels_str = [id2label[l] for l in true_labels]
    preds_str = [id2label[p] for p in preds]
    
    report = classification_report(true_labels_str, preds_str, digits=4)
    print(report)
    
    return report

In [73]:
id2label = {0:'negative', 1:'neutral', 2:'positive'}
report = evaluate_model(trainer, test_dataset, id2label)


              precision    recall  f1-score   support

    negative     1.0000    0.9375    0.9677        16
     neutral     1.0000    1.0000    1.0000         4
    positive     0.9667    1.0000    0.9831        29

    accuracy                         0.9796        49
   macro avg     0.9889    0.9792    0.9836        49
weighted avg     0.9803    0.9796    0.9794        49



## Test

In [74]:
import torch

def predict_aspect_sentiment(aspect, sentence, tokenizer, model, max_length=128):
    """
    aspect: str, từ/cụm từ aspect
    sentence: str, câu chứa aspect
    tokenizer: BERT tokenizer
    model: trained AutoModelForSequenceClassification
    id2label: dict mapping {0: 'negative', 1: 'neutral', 2: 'positive'}
    """
    model.eval()
    
    inputs = tokenizer(
        aspect,
        sentence,
        truncation=True,
        padding='max_length',
        max_length=max_length,
        return_tensors='pt'  
    )

    device = next(model.parameters()).device
    inputs = {k:v.to(device) for k,v in inputs.items()}
    
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits  # [1, 3]
        pred_id = torch.argmax(logits, dim=-1).item() 
    
    return pred_id


In [75]:
id2label = {0:'negative', 1:'neutral', 2:'positive'}

In [76]:
aspect = "battery life"
sentence = "I love the battery life of this laptop, it lasts all day."
pred = predict_aspect_sentiment(aspect, sentence, tokenizer, model)

In [77]:
print(id2label[pred])

positive


In [78]:
aspect = "battery life"
sentence = "I have to charge the battery twice a day, which is really annoying and disappointing."
pred = predict_aspect_sentiment(aspect, sentence, tokenizer, model)
print(id2label[pred])

negative


## Lưu

In [79]:
save_path = "./bert_sentiment_model_saved"

model.save_pretrained(save_path)

tokenizer.save_pretrained(save_path)

('./bert_sentiment_model_saved/tokenizer_config.json',
 './bert_sentiment_model_saved/special_tokens_map.json',
 './bert_sentiment_model_saved/vocab.txt',
 './bert_sentiment_model_saved/added_tokens.json',
 './bert_sentiment_model_saved/tokenizer.json')

# Tạo DEMO 

In [89]:
save_path = "./bert_aspect_model_saved"
from transformers import AutoTokenizer, AutoModelForTokenClassification

model_aspect = AutoModelForTokenClassification.from_pretrained(save_path)
tokenizer_aspect = AutoTokenizer.from_pretrained(save_path)

In [90]:
save_path = "./bert_sentiment_model_saved"
from transformers import AutoModelForSequenceClassification
model_sentiment = AutoModelForSequenceClassification.from_pretrained(save_path)
tokenizer_sentiment = AutoTokenizer.from_pretrained(save_path)

In [91]:
import torch

def predict_aspect(sentence, model, tokenizer):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    
    encoding = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1).squeeze().tolist()
    
    tokens = tokenizer.convert_ids_to_tokens(input_ids.squeeze().cpu())
    
    return list(zip(tokens, predictions))

In [92]:
# Example usage
sentence = "The battery life of this laptop is amazing."
result = predict_aspect(sentence, model_aspect, tokenizer_aspect)

for token, label in result:
    print(f"{token}\t{label}")


[CLS]	0
the	0
battery	1
life	2
of	0
this	0
laptop	0
is	0
amazing	0
.	0
[SEP]	0


In [93]:
import torch

def predict_aspect_sentiment(aspect, sentence, tokenizer, model, max_length=128):
    """
    aspect: str, từ/cụm từ aspect
    sentence: str, câu chứa aspect
    tokenizer: BERT tokenizer
    model: trained AutoModelForSequenceClassification
    {0: 'negative', 1: 'neutral', 2: 'positive'}
    """
    model.eval()
    
    inputs = tokenizer(
        aspect,
        sentence,
        truncation=True,
        padding='max_length',
        max_length=max_length,
        return_tensors='pt'  
    )

    device = next(model.parameters()).device
    inputs = {k:v.to(device) for k,v in inputs.items()}
    
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits  
        pred_id = torch.argmax(logits, dim=-1).item() 
    
    return pred_id


In [94]:
aspect = "battery life"
sentence = "I have to charge the battery twice a day, which is really annoying and disappointing."
pred = predict_aspect_sentiment(aspect, sentence, tokenizer_sentiment, model_sentiment)
print(pred)

0


In [95]:
import gradio as gr
import torch
import re
from transformers import AutoTokenizer, AutoModelForTokenClassification, AutoModelForSequenceClassification

aspect_model_path = "./bert_aspect_model_saved"
sentiment_model_path = "./bert_sentiment_model_saved"

model_aspect = AutoModelForTokenClassification.from_pretrained(aspect_model_path)
tokenizer_aspect = AutoTokenizer.from_pretrained(aspect_model_path)

model_sentiment = AutoModelForSequenceClassification.from_pretrained(sentiment_model_path)
tokenizer_sentiment = AutoTokenizer.from_pretrained(sentiment_model_path)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_aspect.to(device)
model_sentiment.to(device)

# Aspect prediction
def predict_aspect(sentence):
    model_aspect.eval()
    encoding = tokenizer_aspect(sentence, return_tensors="pt", truncation=True, padding=True)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model_aspect(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1).squeeze().tolist()
    
    tokens = tokenizer_aspect.convert_ids_to_tokens(input_ids.squeeze().cpu())
    return list(zip(tokens, predictions))

# Sentiment prediction
def predict_aspect_sentiment(aspect, sentence):
    model_sentiment.eval()
    inputs = tokenizer_sentiment(aspect, sentence, truncation=True, padding='max_length', max_length=128, return_tensors='pt')
    inputs = {k:v.to(device) for k,v in inputs.items()}
    with torch.no_grad():
        outputs = model_sentiment(**inputs)
        logits = outputs.logits
        pred_id = torch.argmax(logits, dim=-1).item()
    return pred_id

def clean_token(token):
    if token in ["[CLS]", "[SEP]"]:
        return ""
    
    if token.startswith("##"):
        return token[2:]
    return token

def highlight_text(text):
    sentences = re.split(r'(?<=[.!?;]) +', text)
    highlighted_paragraph = ""
    
    for sentence in sentences:
        token_preds = predict_aspect(sentence)
        highlighted_sentence = ""
        i = 0
        while i < len(token_preds):
            token, label = token_preds[i]
            token = clean_token(token)
            if not token:
                i += 1
                continue

            if label != 0:
                aspect_tokens = [token]
                j = i + 1
                while j < len(token_preds) and token_preds[j][1] != 0:
                    t, l = token_preds[j]
                    t = clean_token(t)
                    if t: aspect_tokens.append(t)
                    j += 1
                aspect_text = " ".join(aspect_tokens)
                
                sentiment = predict_aspect_sentiment(aspect_text, sentence)
                color = "red" if sentiment == 0 else "orange" if sentiment == 1 else "green"
                
                highlighted_sentence += f'<span style="color:{color}; font-weight:bold">{aspect_text}</span> '
                i = j
            else:
                highlighted_sentence += token + " "
                i += 1
        highlighted_paragraph += highlighted_sentence.strip() + " "
    
    return highlighted_paragraph.strip()

In [96]:

# Gradio interface
iface = gr.Interface(
    fn=highlight_text,
    inputs=gr.Textbox(lines=5, placeholder="Nhập đoạn văn của bạn ở đây..."),
    outputs=gr.HTML(),
    title="ABSA Highlight",
    description="Nhập một đoạn văn, các khía cạnh sẽ được tô màu theo sentiment (red=negative, orange=neutral, green=positive)"
)

iface.launch()

* Running on local URL:  http://127.0.0.1:7861
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://96ed1ea217ef09d37e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
